# One Shot

Ziel ist es, mit einem Request alle Antworten zu erhalten.

In [ ]:
import time
import sqlite3
import response_format
from datetime import datetime
from dotenv import load_dotenv
from litellm import completion

load_dotenv()

connection = sqlite3.connect('../data/survey.db')
cursor = connection.cursor()

In [ ]:
# strategie laden
cursor.execute("SELECT id, system_path, message_path FROM strategies WHERE name = 'oneshot_none'")
strategy_id, system_path, message_path = cursor.fetchone()

# prompts lesen
with open(f"../{system_path}", "r") as f:
    system_prompt = f.read()

with open(f"../{message_path}", "r") as f:
    message_prompt = f.read()

In [ ]:
# fragen laden
cursor.execute("SELECT id, text FROM questions")
questions = cursor.fetchall()

# prompt anreichern
questions_text = "\n".join([f"{id}. {text}" for id, text in questions])
message = message_prompt.replace("{questions}", questions_text)

In [ ]:
# models laden
models_to_test = [
    'Mistral Small',
    'Mistral Large',
    'GPT-5.2 Nano',
    'GPT-5.2',
    'Claude Haikaku',
    'Claude Opus',
    'Gemini Flash',
    'Gemini Pro',
    'Deepseek Chat',
    'Deepseek Reasoner'
]

placeholder = ','.join(['?' for _ in models_to_test])

cursor.execute(
    f"SELECT id, model_id FROM models WHERE name IN ({placeholder})",
    models_to_test
)

models = cursor.fetchall()

In [ ]:
# models befragen und speichern

for model_db_id, model_id in models:

    print(f"{model_id}")

    try:
        # Response Format Exception
        if model_id.startswith("deepseek/"):
            import json
            content = message + "Bitte antworte in diesem JSON Format:" + json.dumps(
                response_format.CompleteResponseFormat.model_json_schema(),
                indent=2
            )
            model_response_format = {"type": "json_object"}
        else:
            content = message
            model_response_format = {
                "type": "json_schema",
                "json_schema": {
                    "name": "survey_response",
                    "schema": response_format.CompleteResponseFormat.model_json_schema()
                }
            }

        # Befragung
        start_time = time.time()
        response = completion(
            model = model_id,
            messages = [
                {"role": "user", "content": content}
            ],
            response_format=model_response_format
        )
        duration_time = time.time() - start_time

        # Run speichern
        cursor.execute(
            "INSERT INTO runs (model_id, strategy_id, prompt_tokens, completion_tokens, duration_time, timestamp) VALUES (?,?,?,?,?,?)",
            (model_db_id, strategy_id, response.usage.prompt_tokens, response.usage.completion_tokens, duration_time, datetime.now().isoformat())
        )
        run_id = cursor.lastrowid

        # Antworten validieren
        parsed_response = response_format.CompleteResponseFormat.model_validate_json(response.choices[0].message.content)

        # Antworten speichern
        for item in parsed_response.answers:
            cursor.execute(
                "INSERT INTO responses (run_id, question_id, answer) VALUES (?,?,?)",
                (run_id, item.question, item.answer)
            )

        connection.commit()
    except Exception as e:
        print(f" ERROR: {e}")
        continue

In [ ]:
connection.close()